In [1]:
import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)
 
# 切换到上一层目录
parent_dir = os.path.dirname(current_dir)
os.chdir(parent_dir)
print("切换后的目录：", parent_dir)

当前工作目录： /Users/lig/Documents/GitHub/NEMESIS/unit_test
切换后的目录： /Users/lig/Documents/GitHub/NEMESIS


In [6]:
import QuantLib as ql
import pandas as pd

from devlib.market.curves.shibor import Shibor3M
from devlib.market.curves.fx_curves import FxForwardCurve, FxImpliedAssetCurve
from devlib.market.volatility.fx_vol_surface import FxVolSurfaceHK

from devlib.utils import ql_calendar_utils
from devlib.utils.ql_date_utils import ql_date, ql_date_str
from devlib.utils.fx_utils import get_calendar

In [3]:
d_ccy = 'CNY'
f_ccy = 'USD'
calendar = get_calendar(f_ccy, d_ccy)

today = ql.Date(25,3,2024)
ql.Settings.instance().evaluationDate = today
spot_f_d = 7.21145

forward_daycount = ql.Actual365Fixed()
forward_calendar = get_calendar(f_ccy, d_ccy)
forward_curve_file = './unit_test/data/USDCNY_curve_data_20240325.xlsx'
forward_curve_data = pd.read_excel(forward_curve_file)
forward_curve_data['SettleDate'] = forward_curve_data['SettleDate'].apply(lambda x: ql_date_str(ql_date(x)))
forward_curve = FxForwardCurve(today, spot_f_d, forward_curve_data, f_ccy, d_ccy, forward_calendar, forward_daycount)

In [4]:
d_curve_file ='./unit_test/data/shibor3m_curve_data_20240325.xlsx'
deposit_mkt_data = pd.read_excel(d_curve_file, sheet_name='deposit')
swap_mkt_data = pd.read_excel(d_curve_file, sheet_name='swap')
fixing_data = pd.read_excel(d_curve_file, sheet_name='fixing')
d_curve = Shibor3M(today, deposit_mkt_data=deposit_mkt_data, swap_mkt_data=swap_mkt_data, fixing_data=fixing_data)

In [5]:
f_curve = FxImpliedAssetCurve(today, d_curve, forward_curve, calendar, daycount=ql.Actual365Fixed(), interpolation_method="dfloglinear", tweak=0.0)

In [7]:
vol_surface_file = './unit_test/data/USDCNY_vol_data_20240325.xlsx'
vol_surface_data = pd.read_excel(vol_surface_file)
vol_surface = FxVolSurfaceHK(today, vol_surface_data, spot_f_d, d_ccy, f_ccy, forward_curve, d_curve, f_curve, forward_calendar, ql.Actual365Fixed())

In [39]:
from nemesis.products.fx.fx_forward_curve import FXForwardCurve
from nemesis.products.fx.fx_implied_curve import FXImpliedAssetCurve
from nemesis.products.fx.fx_vol_surface import FXVolSurface
from nemesis.utils import *
from nemesis.products.rates import *
from nemesis.market.curves import InterpTypes
from nemesis.utils.calendar import JointCalendar

####################################################################
#  NEMESIS ALPHA Version 0.1.0 - This build: 24 Jan 2025 at 10:42 #
####################################################################



In [40]:
value_dt = Date(25, 3, 2024)

curve = FXForwardCurve(value_dt, spot_f_d, forward_curve_data, "USDCNY", JointCalendar([CalendarTypes.CHINA, CalendarTypes.UNITED_STATES]), DayCountTypes.ACT_365F, InterpTypes.FLAT_FWD_RATES)

In [41]:
shibor = QLCurve(value_dt, d_curve, dc_type=DayCountTypes.ACT_360, interp_type=InterpTypes.LINEAR_ZERO_RATES)

In [10]:
shibor.print_table(shibor.pillar_dts)

,Date,ZR,DF
0,2024-03-25,-0.00000,1.000000
1,2024-06-26,2.18297,0.994453
2,2024-09-26,2.16662,0.989079
3,2024-12-26,2.14919,0.983880
4,2025-03-26,2.16796,0.978496
5,2026-03-26,2.17118,0.957449
6,2027-03-26,2.21617,0.935620
7,2028-03-27,2.28361,0.912532
8,2029-03-26,2.36184,0.888499
9,2031-03-26,2.47428,0.840855


In [11]:
forward_curve.get_forward(ql.Date(31,3,2024))

np.float64(7.20853366459192)

In [14]:
curve.get_forward(Date(31,3,2024), DayCountTypes.ACT_365F)

np.float64(7.20853366459192)

In [15]:
forward_curve.curve.discount(ql.Date(31,3,2024))

1.0006059256447029

In [16]:
curve.df(Date(31,3,2024), day_count=DayCountTypes.ACT_365F)

np.float64(1.0006059256447029)

In [62]:
f_curve.curve.discount(ql.Date(1,4,2024))

0.9988767478736454

In [117]:
fx_implied_curve = FXImpliedAssetCurve(value_dt, shibor, curve, JointCalendar([CalendarTypes.CHINA, CalendarTypes.UNITED_STATES]), DayCountTypes.ACT_365F, InterpTypes.FLAT_FWD_RATES)

In [118]:
fx_implied_curve.df(Date(1,4,2024), DayCountTypes.ACT_365F)

np.float64(0.9988767478736454)

In [46]:
vol_surface.interp_vol(ql.Date(19,1,2025), 7.3)

0.05409421171273324

In [120]:
fx_vol_surface = FXVolSurface(value_dt, vol_surface_data, spot_f_d, d_ccy, f_ccy, curve, shibor, fx_implied_curve, cal_type=JointCalendar([CalendarTypes.CHINA, CalendarTypes.UNITED_STATES]), dc_type=DayCountTypes.ACT_365F)

In [121]:
fx_vol_surface.interp_vol(Date(19,1,2025), 7.3)

0.05409421171273324

In [98]:
forward_curve.get_forward(ql.Date(27,6,2024))

np.float64(7.144915)

In [99]:
curve.get_forward(Date(27,6,2024), DayCountTypes.ACT_365F)

np.float64(7.144915)

In [103]:
d_curve.curve.discount(ql.Date(27,6,2024))

0.9943943345745458

In [104]:
shibor.df(Date(27,6,2024), DayCountTypes.ACT_365F)

np.float64(0.994394334574546)

In [114]:
f_curve.curve.discount(ql.Date(27,6,2024))

0.9850187464050563

In [119]:
fx_implied_curve.df(Date(27,6,2024), DayCountTypes.ACT_365F)

np.float64(0.9850187464050563)

In [110]:
d_curve.curve.discount(ql.Date(27,6,2024)) / forward_curve.curve.discount(ql.Date(27,6,2024))

0.9850214919774921

In [112]:
fx_implied_curve.base_curve.df(Date(27,6,2024), DayCountTypes.ACT_365F)

np.float64(0.994394334574546)

In [106]:
fx_implied_curve.forward_curve.df(Date(27,6,2024), DayCountTypes.ACT_365F)

np.float64(1.0095153686223)